In [1]:
import openpyxl as excel

In [2]:
template_file = '/Users/fall_f/Desktop/automation_app/xlsx file/invoice-template.xlsx'
save_file = 'invoice01.xlsx'

name = '田中一郎'
subject = '1月分のご請求'
items = [
    ['りんご', 5, 320],
    ['バナナ', 8, 210],
    ['メロン', 1, 1200]
]

In [3]:
book = excel.load_workbook(template_file)
sheet = book.active
sheet['B4'] = name
sheet['C10'] = subject
total =0
for i, it, in enumerate(items):
    summary, count, price = it
    subtotal = count * price
    total += subtotal
    row = 15 + i
    sheet.cell(row, 2,summary)
    sheet.cell(row, 5, count)
    sheet.cell(row, 6, price)
    sheet.cell(row, 7, subtotal)
sheet['C11'] = total
book.save('save_file.xlsx')

/opt/anaconda3/lib/python3.11/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 請求書!$A:$H.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [1]:
import glob
files = glob.glob('*.xlsx')
print(files)

['save_file.xlsx']


In [10]:
import glob
import openpyxl as excel

target_dir = './salesbooks'
save_file = 'matome2.xlsx'

#ファイルの読み込み処理
def read_files():
    book = excel.Workbook()
    main_sheet = book.active
    enumfiles(main_sheet)
    book.save(save_file)

def enumfiles(main_sheet):
    files =glob.glob(target_dir + '/*.xlsx')
    for fname in files:
        read_book(main_sheet,fname)

def read_book(main_sheet, fname):
    print('read :'  + fname)
    book = excel.load_workbook(fname, data_only= True)
    sheet = book.active
    rows =sheet['A4' : 'F999']
    for row in rows:
        values = [cell.value for cell in row]
        print(values)
        if values[0] is None:
            break
        main_sheet.append(values)

if __name__ == '__main__':
    read_files()


read :./salesbooks/sales-tanaka.xlsx
[datetime.datetime(2020, 3, 3, 0, 0), '上田 祐輝', '豆腐', 4, 1000, 4000]
[datetime.datetime(2020, 3, 5, 0, 0), '松山 竜兵', '豆腐', 1, 1000, 1000]
[datetime.datetime(2020, 3, 5, 0, 0), '大内 圭人', '湯葉', 1, 2000, 2000]
[datetime.datetime(2020, 3, 7, 0, 0), '上田 祐輝', '大豆セット', 2, 2000, 4000]
[datetime.datetime(2020, 3, 15, 0, 0), '松山 竜兵', '小豆', 1, 1000, 1000]
[datetime.datetime(2020, 3, 16, 0, 0), '大内 圭人', '豆腐', 4, 1000, 4000]
[datetime.datetime(2020, 3, 20, 0, 0), '松山 竜兵', '小豆', 5, 1000, 5000]
[None, None, None, None, None, None]
read :./salesbooks/sales-kakei.xlsx
[datetime.datetime(2020, 3, 6, 0, 0), '上田 祐輝', '醤油', 20, 1500, 30000]
[datetime.datetime(2020, 3, 11, 0, 0), '松山 竜兵', 'ソース', 2, 1500, 3000]
[datetime.datetime(2020, 3, 20, 0, 0), '上田 祐輝', '味噌', 8, 950, 7600]
[datetime.datetime(2020, 3, 21, 0, 0), '松山 竜兵', '醤油', 10, 1500, 15000]
[None, None, None, None, None, None]
read :./salesbooks/sales-ooda.xlsx
[datetime.datetime(2020, 3, 6, 0, 0), '鈴木 太郎', 'ミカンゼリー', 

In [11]:
dummy_data = [
    ['伊藤', 300],
    ['伊藤', 600],
    ['伊藤', 200],
    ['田中', 300],
    ['田中', 200]
]

users = {}
for row in dummy_data:
    name, value = row
    if name not in users:
        users[name] = []
    users[name].append(row)

for name, rows in users.items():
    print(rows)
    total = 0
    for row in rows :
        total += row[1]
    print(name, total)


[['伊藤', 300], ['伊藤', 600], ['伊藤', 200]]
伊藤 1100
[['田中', 300], ['田中', 200]]
田中 500


In [17]:
users  = {}
for row in dummy_data:
    name , value = row
    if name not in users:
        users[name] = value
    else:
        users[name] += value

In [10]:
import openpyxl as excel
import json
from datetime import datetime

in_file = '/Users/fall_f/Desktop/automation_app/ch3/matome.xlsx'
out_file = 'matome.json'

def split_list(in_file):
    users = read_and_split(in_file)
    result = {}
    for name, rows in users.items():
        result[name] = calc_users(rows)
        print(name, result[name]['total'])

    # datetime を含まない result を保存（users ではなく result を保存）
    with open(out_file, 'wt', encoding='utf-8') as fp:
        json.dump(result, fp, ensure_ascii=False, indent=2)

# 顧客ごとにデータを分割
def read_and_split(in_file):
    users = {}
    book = excel.load_workbook(in_file)
    sheet = book.active
    for row in sheet.iter_rows():
        values = [cell.value for cell in row]
        name = values[1]
        if name is None: break
        if name not in users:
            users[name] = []
        users[name].append(values)
    return users

def calc_users(rows):
    total = 0
    items = []
    for row in rows:
        date, _, item, cnt, price, _ = row
        # datetime を文字列に変換（JSON化のため）
        date_s = date.strftime('%m/%d') if isinstance(date, datetime) else str(date)
        items.append([date_s, item, cnt, price])
        total += cnt * price
    return {'items': items, 'total': total}

if __name__ == '__main__':
    split_list(in_file)

上田 祐輝 45600
松山 竜兵 40000
大内 圭人 6000
鈴木 太郎 33000
伊藤 次郎 10500
西川 三郎 19100
山田 史朗 34000
山下 五郎 5700
小島 重吉 14000


In [14]:
import os
in_file  = 'matome.json'
template_file = 'invoice-template.xlsx'
subject = '2月分のご請求'
out_dir = './ch3'
def gen_invoice():
    with open(in_file, 'rt') as fp:
        users = json.load(fp)
    for name , data in users.items():
        make_users_invoice(name, data)

def make_users_invoice(name, data):
    book = excel.load_workbook(template_file)
    sheet = book.active

    sheet['B4'] = name
    sheet['C10'] = subject
    sheet['C11'] = data['total']

    for i, it in enumerate(data['items']):
        date, summary, cnt, price = it
        row = 15 + i
        sheet.cell(row, 2, f"{summary} ({date})")
        sheet.cell(row, 5, cnt)
        sheet.cell(row, 6, price)
        sheet.cell(row, 7, cnt * price)

    # 🔧 ディレクトリを作成（なければ）
    os.makedirs(out_dir, exist_ok=True)

    out_file = f"{out_dir}/{name}様.xlsx"
    book.save(out_file)
    print('保存完了 : ' + out_file)

if __name__ == '__main__':
    gen_invoice()

保存完了 : ./ch3/上田 祐輝様.xlsx
保存完了 : ./ch3/松山 竜兵様.xlsx
保存完了 : ./ch3/大内 圭人様.xlsx
保存完了 : ./ch3/鈴木 太郎様.xlsx
保存完了 : ./ch3/伊藤 次郎様.xlsx
保存完了 : ./ch3/西川 三郎様.xlsx
保存完了 : ./ch3/山田 史朗様.xlsx
保存完了 : ./ch3/山下 五郎様.xlsx
保存完了 : ./ch3/小島 重吉様.xlsx


/opt/anaconda3/lib/python3.11/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 請求書!$A:$H.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [15]:
import openpyxl as excel
book = excel.load_workbook('/Users/fall_f/Desktop/automation_app/ch3/date-sample.xlsx')
sheet = book.active
sheet['A4'].number_format
sheet['A4'].value

/opt/anaconda3/lib/python3.11/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '3月'!$A:$C.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


datetime.datetime(2020, 3, 3, 0, 0)

In [18]:
import openpyxl as excel
import datetime

in_file = 'date-sample.xlsx'
out_file = 'date_wareki.xlsx'
cell_format = '[$-ja-JP]ggge"年"m"月"d"日"'

def date_wareki(in_file, out_file):
    book = excel.load_workbook(in_file)
    sheet = book.active
    for sheet in book.worksheets:
        for row in sheet.iter_rows():
            for cell in row:
                check_cell(cell)
    book.save(out_file)

def check_cell(cell):
    if type(cell.value) is datetime.datetime :
        cell.number_format = cell_format

if __name__ == '__main__':
    date_wareki(in_file, out_file)

In [23]:
import openpyxl as excel 
from openpyxl.styles import PatternFill
import datetime

in_file = 'date-sample.xlsx'
out_file = 'date-mark_olditems.xlsx'
limit = datetime.datetime(2020, 1,1)

def date_mark(in_file, out_file):
    book = excel.load_workbook(in_file)
    for sheet in book.worksheets:
        for row in sheet.iter_rows(min_row=4):
            check_row(row)
    book.save(out_file)

def check_row(row):
    date = row[0].value
    if type(date) is not datetime.datetime:
        return
    
    if date < limit:
        red = PatternFill(
            fill_type = 'solid',
            fgColor = 'ff0000'
        )
        for cell in row:
            cell.fill = red
    
if __name__ == '__main__' :
    date_mark(in_file, out_file)


In [28]:
import openpyxl as excel

in_file = 'name2.xlsx'
out_file = 'seimei1.xlsx'

in_book = excel.load_workbook(in_file)
in_sheet = in_book.active

out_book = excel.Workbook()
out_sheet = out_book.active

for row in in_sheet.iter_rows():
    sei = row[0].value
    nam = row[1].value
    seimei = sei + nam
    out_sheet.append([seimei])

out_book.save(out_file)
